In [1]:
import pandas as pd
import numpy as np
from sklearn import model_selection, preprocessing, pipeline
from sklearn.ensemble import RandomForestRegressor
import psycopg2
from sqlalchemy import create_engine
from pins import board_rsconnect

In [2]:
from sqlalchemy import create_engine
engine = create_engine(
    "postgresql+psycopg2://content:@soleng-content-rds1.cloiraotshw4.us-east-1.rds.amazonaws.com/rds",
    isolation_level="SERIALIZABLE",
)

In [3]:
sql = "Select * From bike_model_data "

In [6]:
SQL_for_file_output = "COPY ({0}) TO STDOUT WITH CSV HEADER".format(sql)

conn = psycopg2.connect(dbname="rds", user="content", password="", host="")
db_cursor = conn.cursor()

In [14]:
# Set up a variable to store our file path and name.
t_path_n_file = "./all-dates.csv"

# Trap errors for opening the file
try:
    with open(t_path_n_file, 'w') as f_output:
        db_cursor.copy_expert(SQL_for_file_output, f_output)
except psycopg2.Error as e:
    t_message = "Error: " + e + "/n query we ran: " + s + "/n t_path_n_file: " + t_path_n_file
    print(f'{"error.html" + t_message}')

# Success!

# Clean up: Close the database cursor and connection
db_cursor.close()

In [15]:
conn.close()

In [2]:
df_all_dates = pd.read_csv("all-dates.csv")

In [3]:
df_all_dates.shape

(12260589, 8)

In [21]:
df_all_dates.head()

,id,hour,date,month,dow,n_bikes,lat,lon
0,73,4,2021-10-03,10,Sunday,2.0,38.933668,-76.991016
1,73,4,2021-10-04,10,Monday,8.0,38.933668,-76.991016
2,73,4,2021-10-05,10,Tuesday,7.0,38.933668,-76.991016
3,73,4,2021-10-06,10,Wednesday,4.0,38.933668,-76.991016
4,73,4,2021-10-07,10,Thursday,4.0,38.933668,-76.991016


In [12]:
import os

In [15]:
os.environ['CONNECT_API_KEY']

KeyError: 'CONNECT_API_KEY'

In [17]:
board = board_rsconnect(server_url="https://colorado.rstudio.com/rsc/", api_key="fill_your_own")
#> Connecting to RSC 1.9.0.1 at <https://connect.rstudioservices.com>

In [8]:
board.pin_write?

Signature: board.pin_write(*args, access_type=None, **kwargs)
Docstring:
Write a pin.

Extends parent method in the following ways:

* Modifies content item to include any title and description changes.
* Adds access_type argument to specify who can see content. Defaults to "acl".


Parent method documentation:

Write a pin object to the board.

Parameters
----------
x:
    An object (e.g. a pandas DataFrame) to pin.
name:
    Pin name.
type:
    File type used to save ``x`` to disk.
title:
    A title for the pin; most important for shared boards so that others
    can understand what the pin contains. If omitted, a brief description
    of the contents will be automatically generated.
description:
    A detailed description of the pin contents.
metadata:
    A dictionary containing additional metadata to store with the pin.
    This gets stored on the Meta.user field.
versioned:
    Whether the pin should be versioned.
created:
    A date to store in the Meta.created field. This fiel

In [18]:
board.pin_write(
    df_all_dates,
    name="xu.fei/bike-all-dates-05042022", 
    title="Shared Bike Data", 
    description="Shared bike data until May 4th, 2022",
    type="csv")
#> Writing to pin 'hadley/sales-summary'

Writing to pin 'xu.fei/bike-all-dates-05042022'


Meta(title='Shared Bike Data', description='Shared bike data until May 4th, 2022', created='20220505T164106Z', pin_hash='802fb4641d9829b0', file='bike-all-dates-05042022.csv', file_size=689184111, type='csv', api_version=1, version=VersionRaw(version='56032'), name='xu.fei/bike-all-dates-05042022', user={})

In [16]:
! ls -alh

total 639M
drwx--x---  6 xu.fei xu.fei 4.0K May  4 21:55 .
drwxr-xr-x 59 xu.fei 150420 4.0K May  4 20:24 ..
-rw-------  1 xu.fei xu.fei 639M May  4 21:53 all-dates.csv
-rw-------  1 xu.fei xu.fei  885 Apr 25 22:18 app.py
-rw-------  1 xu.fei xu.fei  28K May  4 21:36 bike_shiny.ipynb
-rw-------  1 xu.fei xu.fei 1.3K Apr 28 13:42 bike_shiny.py
-rw-------  1 xu.fei xu.fei  102 Apr 27 21:32 .env
drwx--x---  2 xu.fei xu.fei  106 May  4 21:36 .ipynb_checkpoints
drwx--x---  2 xu.fei xu.fei   65 Apr 27 23:13 __pycache__
-rw-------  1 xu.fei xu.fei 1.5K Apr 27 22:24 requirements.txt
drwx--x---  2 xu.fei xu.fei   31 Apr 25 18:49 rsconnect-python
-rw-------  1 xu.fei xu.fei 3.5K May  4 21:55 Untitled.ipynb
drwx--x---  6 xu.fei xu.fei   87 Apr 25 18:25 .venv


In [25]:
!pwd

/usr/home/xu.fei/py-shiny-app


In [ ]:
df_all_dates = pd.read_sql_query(sql,engine)